In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

In [ ]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NY_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # no nulls in coords
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # no nulls in fips

In [5]:
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []
i=0
for i in tqdm(range(len(lat))):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()

        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('36')] #NY
            fips_elements =[element for element in startswith if len(element)>=15]
            try:
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()

        i=i+1

  0%|          | 0/104 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

104


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_6120/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
285,3951297402,NY,"221 E 71st Street 0.0, New York, NY",40.794403,-74.005440,[]
420,8426367110,NY,"150 E. 58th Street 39th Floor, NEW YORK, NY",40.784798,-74.011215,[]
583,3057417102,NY,"156 W 56th St 0.0, New York, NY",40.783627,-74.012220,[]
1209,3957807107,NY,"400 COLUMBUS AVE, VALHALLA, NY",41.098358,-73.775792,[361190121013033]
2619,2306618706,NY,"45 BROADWAY FL 27, NEW YORK, NY",40.706470,-74.013041,[360610013003004]
...,...,...,...,...,...,...
70514,9678357108,NY,"16 MAYFAIR LN, BUFFALO, NY",42.903020,-78.875233,[360290067024002]
70756,6886047110,NY,"252 Chestnut Ridge Rd, SARATOGA SPRINGS, NY",41.029201,-74.074111,[]
71674,4142937104,NY,"2537 STATE ROUTE 9, BALLSTON SPA, NY",33.679340,-85.675174,[]
71957,3822928510,NY,"384 SINGINGWOOD DR, HOLBROOK, NY",40.797592,-73.059395,[361031466141002]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_6120/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
285,3951297402,NY,"221 E 71st Street 0.0, New York, NY",40.794403,-74.005440,NaN
420,8426367110,NY,"150 E. 58th Street 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
583,3057417102,NY,"156 W 56th St 0.0, New York, NY",40.783627,-74.012220,NaN
1209,3957807107,NY,"400 COLUMBUS AVE, VALHALLA, NY",41.098358,-73.775792,361190121013033
2619,2306618706,NY,"45 BROADWAY FL 27, NEW YORK, NY",40.706470,-74.013041,360610013003004
...,...,...,...,...,...,...
70514,9678357108,NY,"16 MAYFAIR LN, BUFFALO, NY",42.903020,-78.875233,360290067024002
70756,6886047110,NY,"252 Chestnut Ridge Rd, SARATOGA SPRINGS, NY",41.029201,-74.074111,NaN
71674,4142937104,NY,"2537 STATE ROUTE 9, BALLSTON SPA, NY",33.679340,-85.675174,NaN
71957,3822928510,NY,"384 SINGINGWOOD DR, HOLBROOK, NY",40.797592,-73.059395,361031466141002


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
285,3951297402,NY,"221 E 71st Street 0.0, New York, NY",40.794403,-74.005440,NaN
420,8426367110,NY,"150 E. 58th Street 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
583,3057417102,NY,"156 W 56th St 0.0, New York, NY",40.783627,-74.012220,NaN
3781,3060367101,NY,"150 E. 58th St. 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
4272,5472187210,NY,"250 W 55TH ST, 35TH FL, NEW YORK, NY",40.784020,-74.014680,NaN
...,...,...,...,...,...,...
68700,8903858605,NY,"1066 State Route 9, Queensbury, NY",33.781712,-85.691415,NaN
69699,8854587309,NY,"2373 ASP ROUTE 1 Suite 1, SALAMANCA, NY",37.226692,-77.401370,NaN
70756,6886047110,NY,"252 Chestnut Ridge Rd, SARATOGA SPRINGS, NY",41.029201,-74.074111,NaN
71674,4142937104,NY,"2537 STATE ROUTE 9, BALLSTON SPA, NY",33.679340,-85.675174,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
285,3951297402,NaN
420,8426367110,NaN
583,3057417102,NaN
1209,3957807107,361190121013033
2619,2306618706,360610013003004
...,...,...
70514,9678357108,360290067024002
70756,6886047110,NaN
71674,4142937104,NaN
71957,3822928510,361031466141002


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NY_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,603583,1009208907,NY,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,3.610595e+14
1,603584,1029567410,NY,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,3.606101e+14
2,603585,1040387100,NY,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,3.608100e+14
3,603586,1060377105,NY,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,3.606101e+14
4,603587,1100587104,NY,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,3.606100e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,603583,1009208907,NY,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,3.610595e+14,NaN
1,603584,1029567410,NY,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,3.606101e+14,NaN
2,603585,1040387100,NY,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,3.608100e+14,NaN
3,603586,1060377105,NY,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,3.606101e+14,NaN
4,603587,1100587104,NY,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,3.606100e+14,NaN
...,...,...,...,...,...,...,...,...
73814,677397,9808878607,NY,"13159 224th St, Laurelton, NY",40.679641,-73.744327,3.608106e+14,NaN
73815,677398,9818628508,NY,"519 Central Ave, Massapequa, NY",40.678092,-73.471767,3.605952e+14,NaN
73816,677399,9905798609,NY,"409 E State St, Albion, NY",43.246697,-78.183473,3.607304e+14,NaN
73817,677400,9985928403,NY,"1175 W Broadway, Hewlett, NY",40.637690,-73.705559,3.605941e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,603583,1009208907,NY,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,3.610595e+14,NaN,361059515002013.0
1,603584,1029567410,NY,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,3.606101e+14,NaN,360610082002000.0
2,603585,1040387100,NY,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,3.608100e+14,NaN,360810019031008.0
3,603586,1060377105,NY,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,3.606101e+14,NaN,360610061003001.0
4,603587,1100587104,NY,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,3.606100e+14,NaN,360610013003001.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1009208907,NY,"641 Old Route 17, Monticello, NY",41.701466,-74.735090,361059515002013.0
1,1029567410,NY,"60 EAST 42ND ST SUITE 1250, NEW YORK, NY",40.752097,-73.978733,360610082002000.0
2,1040387100,NY,"1102 QUEENS PLZ S, LONG ISLAND CITY, NY",40.752814,-73.946568,360810019031008.0
3,1060377105,NY,"853 Broadway, Suite 606 0.0, New York, NY",40.734690,-73.990790,360610061003001.0
4,1100587104,NY,"101 Greenwich Street, 4th Floor 0.0, New York, NY",40.708027,-74.013560,360610013003001.0
...,...,...,...,...,...,...
73814,9808878607,NY,"13159 224th St, Laurelton, NY",40.679641,-73.744327,360810630001001.0
73815,9818628508,NY,"519 Central Ave, Massapequa, NY",40.678092,-73.471767,360595212002021.0
73816,9905798609,NY,"409 E State St, Albion, NY",43.246697,-78.183473,360730407021012.0
73817,9985928403,NY,"1175 W Broadway, Hewlett, NY",40.637690,-73.705559,360594117001003.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
285,3951297402,NY,"221 E 71st Street 0.0, New York, NY",40.794403,-74.005440,NaN
420,8426367110,NY,"150 E. 58th Street 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
583,3057417102,NY,"156 W 56th St 0.0, New York, NY",40.783627,-74.012220,NaN
3781,3060367101,NY,"150 E. 58th St. 39th Floor, NEW YORK, NY",40.784798,-74.011215,NaN
4272,5472187210,NY,"250 W 55TH ST, 35TH FL, NEW YORK, NY",40.784020,-74.014680,NaN
...,...,...,...,...,...,...
68700,8903858605,NY,"1066 State Route 9, Queensbury, NY",33.781712,-85.691415,NaN
69699,8854587309,NY,"2373 ASP ROUTE 1 Suite 1, SALAMANCA, NY",37.226692,-77.401370,NaN
70756,6886047110,NY,"252 Chestnut Ridge Rd, SARATOGA SPRINGS, NY",41.029201,-74.074111,NaN
71674,4142937104,NY,"2537 STATE ROUTE 9, BALLSTON SPA, NY",33.679340,-85.675174,NaN


Convert to File

In [19]:
fips_NY = fips_merge_drop

In [20]:
fips_NY.to_csv("../../../data/state_data/geo/geo_fips/23/NY_fips_scraped.csv") 